## Importing all libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import svm
import scipy.stats as stats
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Importing all csv & merging them

In [2]:
pd.set_option('display.max_columns', 100) # shows all the columns

# loading all the datasets
features_0 = pd.read_csv('/Users/suborna/Github/Capstone_Project/Data/Combined_Provider.csv')
features_2 = pd.read_csv('/Users/suborna/Github/Capstone_Project/Data/new_features.csv')
# dropping because this column already exist in features_2, we don't need duplicate
features_1 = features_0.drop('PotentialFraud', axis = 1) 

# merging features_1 & features_2 to have the complete dataset after feature engineering
All_Features = features_2.merge(features_1, on = 'Provider')

# setting index as provider for ML modeling
Train_set = features_2.merge(features_1, on = 'Provider').set_index('Provider')

# replacing categories with ordinal data 
Train_set['PotentialFraud'] = Train_set['PotentialFraud'].replace(['Yes', 'No'], [1,0])

# seperating complete dataset into 2 for plotting
fraud_df = All_Features.loc[All_Features.PotentialFraud == 'Yes']
Fraud_df = fraud_df.reset_index().rename(columns = {'index': 'Index'})
nfraud_df = All_Features.loc[All_Features.PotentialFraud == 'No']
NFraud_df = nfraud_df.reset_index().rename(columns = {'index': 'Index'})


In [3]:
All_Features.shape

(5410, 72)

In [4]:
# check the distribution
np.round(Train_set.PotentialFraud.value_counts()/All_Features.shape[0], 2)

0    0.91
1    0.09
Name: PotentialFraud, dtype: float64

In [5]:
Train_set.sample(5)

,Male_Patient,Female_Patient,Age(26-40),Age(41-60),Age(61-80),Age(81-100),Race_White,Race_Black,Race_Hispanic,Race_Native,Hos_Stay_Total,Payment_Total,Claim_Len_Total,Alzheimer,Heartfailure,KidneyDisease,Cancer,ObstrPulmonary,Depression,Diabetes,IschemicHeart,Osteoporasis,rheumatoidarthritis,stroke,RenalDisease,State_Count,County_Count,DiagGroupCode,PotentialFraud,CDC_2449,CDC_25000,CDC_2720,CDC_2724,CDC_4011,CDC_4019,CDC_41401,CDC_42731,CDC_4280,CDC_496,CDC_53081,CDC_5990,CDC_V5861,CDC_V5869,CPC_2724,CPC_3722,CPC_3893,CPC_3995,CPC_4019,CPC_4516,CPC_66,CPC_8151,CPC_8154,CPC_9904,DGC_166,DGC_167,DGC_183,DGC_186,DGC_187,DGC_188,DGC_192,DGC_202,DGC_208,DGC_881,DGC_882,DGC_883,DGC_884,DGC_887,DGC_939,DGC_940,DGC_941,DGC_945
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51490,23.0,33.0,0.0,8.0,33.0,15.0,39.0,4.0,8.0,5.0,235.0,493334.0,259,36,42,30,9,30,39,52,53,21,25,11,17,3,9,46.0,1,2,5,0.0,6,0.0,8,5.0,4,2,5.0,5.0,6.0,0.0,0.0,1,1,1,1,1,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
PRV56077,2.0,5.0,0.0,2.0,3.0,2.0,7.0,0.0,0.0,0.0,0.0,2450.0,40,3,4,4,3,0,4,6,5,3,0,0,3,1,1,0.0,0,1,1,0.0,0,1.0,0,0.0,0,0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV54947,4.0,20.0,1.0,2.0,14.0,7.0,19.0,4.0,0.0,1.0,0.0,7680.0,22,9,13,8,8,7,7,12,17,8,6,1,6,2,8,0.0,0,1,1,0.0,1,0.0,2,0.0,0,0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV56871,5.0,6.0,0.0,0.0,8.0,3.0,9.0,2.0,0.0,0.0,0.0,4400.0,20,3,7,4,2,4,6,6,9,5,1,2,3,1,1,0.0,0,1,2,0.0,0,1.0,2,0.0,0,0,0.0,0.0,0.0,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV57501,23.0,40.0,1.0,7.0,34.0,21.0,58.0,5.0,0.0,0.0,0.0,11970.0,17,22,31,19,13,14,26,46,45,15,22,2,11,4,12,0.0,0,4,2,3.0,3,1.0,5,0.0,4,2,0.0,0.0,0.0,1.0,4.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Running baseline model (Logistic Regression)

In [87]:
# define class weights
w = {0:9, 1:91}

# Create instance of Logistic Regression Class
lm = LogisticRegression(class_weight = 'balanced', solver = 'liblinear', random_state = 42)

In [90]:
x = Train_set.drop('PotentialFraud', axis = 1)
y = Train_set['PotentialFraud']

X = preprocessing.scale(x) # scaling the X variables
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size = 0.3)

lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

In [91]:
lm.score(X_train, y_train)

0.9992078162133615

In [92]:
lm.score(X_test, y_test)

0.9969192852741836

In [93]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print('Classification Report') 
print(classification_report(y_test, y_pred))
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))

Confusion Matrix
[[1443    1]
 [   4  175]]
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1444
           1       0.99      0.98      0.99       179

    accuracy                           1.00      1623
   macro avg       1.00      0.99      0.99      1623
weighted avg       1.00      1.00      1.00      1623

Accuracy Score
0.9969192852741836


In [95]:
Features = pd.DataFrame(np.column_stack(lm.coef_ ), index = x.columns, columns = ['Coefficients'])
Features.loc[Features.Coefficients > 0].sort_values(ascending = False, by = 'Coefficients')

,Coefficients
CDC_496,3.458879
CDC_5990,3.257098
CDC_53081,3.005338
CDC_41401,2.804348
Age(26-40),1.316515
State_Count,1.075006
CDC_4019,0.910069
DGC_202,0.733904
DGC_192,0.684615
Age(81-100),0.627350


# Log Regression (Random Forest Feature Selection)

In [97]:
reduced_features = Train_set.drop(['RenalDisease', 'DGC_945', 'CPC_8151', 'DGC_939', 'DGC_187', 'DGC_167',
       'DGC_940', 'Cancer', 'CPC_4516', 'DGC_883', 'Race_Black', 'CPC_3722',
       'Race_White', 'Age(26-40)', 'Race_Hispanic', 'Race_Native', 'CPC_3893',
       'rheumatoidarthritis', 'State_Count', 'Depression', 'CPC_8154',
       'Age(81-100)', 'CPC_9904', 'DGC_166', 'DGC_183', 'DGC_186', 'DGC_188',
       'DGC_192', 'DGC_202', 'DGC_208', 'DGC_881', 'Age(61-80)', 'DGC_884',
       'DGC_887', 'DGC_941', 'Age(41-60)', 'DGC_882'], axis = 1)

# train/test/split
x = reduced_features.drop('PotentialFraud', axis = 1)
X = preprocessing.scale(x)
y = reduced_features['PotentialFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle = True, test_size = 0.3)

lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

In [98]:
print(lm.score(X_train, y_train))
lm.score(X_test, y_test)

0.9989437549511486


0.9975354282193469

In [99]:
print(np.round(lm.coef_ , 3))
print(' * ' * 30)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print(' * ' * 30)
print('Classification Report') 
print(classification_report(y_test, y_pred))
print(' * ' * 30)
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))

[[ 4.340e-01  1.286e+00  4.320e-01  7.300e-01 -2.200e-02  5.860e-01
   6.600e-01  3.880e-01  5.050e-01  4.720e-01  5.440e-01  3.340e-01
   5.640e-01  1.040e-01 -8.480e-01  2.840e-01  2.690e-01 -2.062e+00
  -1.170e-01 -1.800e+00  9.880e-01  3.729e+00  2.000e-03 -7.470e-01
   3.505e+00  3.070e+00  3.717e+00 -1.731e+00 -1.775e+00  4.300e-02
   8.800e-02 -3.000e-02  7.500e-02]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Confusion Matrix
[[1444    0]
 [   4  175]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1444
           1       1.00      0.98      0.99       179

    accuracy                           1.00      1623
   macro avg       1.00      0.99      0.99      1623
weighted avg       1.00      1.00      1.00      1623

 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 

In [102]:
Features = pd.DataFrame(np.column_stack(lm.coef_ ), index = x.columns, columns = ['Coefficients'])
Features.loc[Features.Coefficients > 0].sort_values(ascending = False, by = 'Coefficients')

,Coefficients
CDC_41401,3.729281
CDC_5990,3.716896
CDC_496,3.504576
CDC_53081,3.070251
Female_Patient,1.285980
CDC_4019,0.988120
Payment_Total,0.729674
Heartfailure,0.659910
Alzheimer,0.585790
stroke,0.563605


# Logistic Regression with Lasso Penalty (Feature Selection Process)

In [135]:
x = Train_set.drop('PotentialFraud', axis = 1)
y = Train_set['PotentialFraud']

X = preprocessing.scale(x) # scaling the X variables
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size = 0.3)

In [138]:
w = {0:10, 1:90}
lm_lasso = LogisticRegression(solver = 'liblinear', penalty = 'l1', class_weight = 'balanced')
lm_lasso.fit(X_train, y_train)
y_pred = lm_lasso.predict(X_test)

In [137]:
print('Train Model Score')
print(lm_lasso.score(X_train, y_train))
print(' * '* 30)
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))
print(' * ' * 30)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print(' * ' * 30)
print('Classification Report') 
print(classification_report(y_test, y_pred))
print(' * ' * 30)

Train Model Score
0.9992078162133615
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Accuracy Score
0.9969192852741836
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Confusion Matrix
[[1443    1]
 [   4  175]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1444
           1       0.99      0.98      0.99       179

    accuracy                           1.00      1623
   macro avg       1.00      0.99      0.99      1623
weighted avg       1.00      1.00      1.00      1623

 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


In [140]:
Features = pd.DataFrame(np.column_stack(lm_lasso.coef_ ), index = x.columns, columns = ['Coefficients'])
Features.loc[Features.Coefficients > 0].sort_values(ascending = False, by = 'Coefficients')

,Coefficients
CDC_41401,6.187871
CDC_496,5.065089
CDC_5990,5.058769
CDC_53081,4.375149
Age(26-40),3.034713
State_Count,1.308091
Age(81-100),1.283769
Female_Patient,0.982720
CDC_4019,0.775491
DGC_202,0.558643


In [24]:
# grid_params = {'C' : [0.001,0.01,0.1,1,10,100]}
# grid_search_lm = GridSearchCV(lm_lasso, grid_params, cv = 3, scoring = 'accuracy', n_jobs = -1)
# %time grid_search_lm.fit(X, y)

In [25]:
# grid_search_lm.best_params_

In [26]:
# grid_search_lm.best_score_

In [27]:
# y_Pred = grid_search_lm.predict(X_test)
# print(accuracy_score(y_test, y_Pred))

# Logistic Regression with New Features from Lasso Penalty

In [166]:
x = Train_set[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']]

# x = Train_set[['CDC_2724', 'CDC_41401', 'CDC_496', 'CDC_53081', 'CDC_5990']]
               
y = Train_set['PotentialFraud']

X = preprocessing.scale(x) # scaling the X variables
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size = 0.4)

In [167]:
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

print('Train Model Score')
print(lm.score(X_train, y_train))
print(' * '* 30)
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))
print(' * ' * 30)
print(' * ' * 30)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print(' * ' * 30)
print('Classification Report') 
print(classification_report(y_test, y_pred))

Train Model Score
0.9987677141096735
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Accuracy Score
0.9981515711645101
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Confusion Matrix
[[1934    0]
 [   4  226]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1934
           1       1.00      0.98      0.99       230

    accuracy                           1.00      2164
   macro avg       1.00      0.99      1.00      2164
weighted avg       1.00      1.00      1.00      2164



In [143]:
Features = pd.DataFrame(np.column_stack(lm.coef_ ), index = x.columns, columns = ['Coefficients'])
Features

,Coefficients
CDC_41401,5.511437
CDC_496,3.814458
CDC_5990,4.750317
CDC_53081,3.235203


# Random Forest with Lasso Features

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [168]:
x = Train_set[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']]

y = Train_set['PotentialFraud']

X = preprocessing.scale(x) # scaling the X variables
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size = 0.3)

rf = RandomForestClassifier(class_weight = w, max_depth = 3, max_features = 1, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 20, random_state = 42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [169]:
print('Train Model Score')
print(rf.score(X_train, y_train))
print(' * '* 30)
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))
print(' * ' * 30)
print(' * ' * 30)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print(' * ' * 30)
print('Classification Report') 
print(classification_report(y_test, y_pred))

Train Model Score
0.9960390810668075
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Accuracy Score
0.9956869993838571
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Confusion Matrix
[[1444    0]
 [   7  172]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1444
           1       1.00      0.96      0.98       179

    accuracy                           1.00      1623
   macro avg       1.00      0.98      0.99      1623
weighted avg       1.00      1.00      1.00      1623



# Gradient Boosting with Lasso Features

# Support Vector Machines

In [187]:
svm_model = svm.SVC(kernel='linear', C=1e5, class_weight = 'balanced', random_state = 42)

x = Train_set[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']]

y = Train_set['PotentialFraud']

X = preprocessing.scale(x) # scaling the X variables
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True, test_size = 0.3)
svm_model.fit(X_test, y_test)
y_pred = svm_model.predict(X_test)

In [188]:
svm_index = svm_model.n_support_ # index of support vector
print('Index of support vector: ' + str(svm_index))

Index of support vector: [20  4]


In [189]:
print('Train Model Score')
print(svm_model.score(X_train, y_train))
print(' * '* 30)
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))
print(' * ' * 30)
print(' * ' * 30)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print(' * ' * 30)
print('Classification Report') 
print(classification_report(y_test, y_pred))

Train Model Score
0.9981515711645101
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Accuracy Score
0.9993838570548367
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Confusion Matrix
[[1444    0]
 [   1  178]]
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1444
           1       1.00      0.99      1.00       179

    accuracy                           1.00      1623
   macro avg       1.00      1.00      1.00      1623
weighted avg       1.00      1.00      1.00      1623



In [190]:
Features = pd.DataFrame(np.column_stack(svm_model.coef_ ), index = x.columns, columns = ['Coefficients'])
Features.sort_values(ascending = False, by = 'Coefficients')

,Coefficients
CDC_53081,3.498716e+00
CDC_5990,3.334599e+00
Age(81-100),1.025562e+00
CDC_41401,9.042206e-01
CDC_496,5.009841e-01
State_Count,4.283352e-04
CPC_66,6.552425e-11
CPC_3995,2.786860e-11
DGC_202,1.598877e-11
DGC_188,1.567302e-11


### Ploting the new features (EDA)

In [191]:
Fraud_df[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']].describe()

,CDC_41401,CDC_496,CDC_5990,CDC_53081,Age(26-40),State_Count,Age(81-100),Female_Patient,CDC_4019,DGC_202,DGC_192,DGC_188,CPC_66,CPC_3995
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,5.053360,4.415020,4.484190,4.626482,6.494071,5.379447,70.411067,139.179842,7.197628,0.187747,0.193676,0.191700,0.478261,0.446640
std,2.754496,2.713679,2.741819,2.681216,9.290950,5.430961,101.843041,198.716638,2.272625,0.473378,0.493562,0.508161,0.503966,0.497637
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,2.000000,2.000000,1.000000,2.000000,14.000000,29.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,4.000000,5.000000,5.000000,3.000000,4.000000,34.000000,67.500000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,7.000000,7.000000,7.000000,7.750000,6.000000,76.750000,146.750000,9.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,82.000000,38.000000,876.000000,1641.000000,10.000000,2.000000,4.000000,3.000000,2.000000,1.000000


In [193]:
Fraud_df[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']].median()

CDC_41401          5.0
CDC_496            4.0
CDC_5990           5.0
CDC_53081          5.0
Age(26-40)         3.0
State_Count        4.0
Age(81-100)       34.0
Female_Patient    67.5
CDC_4019           8.0
DGC_202            0.0
DGC_192            0.0
DGC_188            0.0
CPC_66             0.0
CPC_3995           0.0
dtype: float64

In [192]:
NFraud_df[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']].sample(500).describe()

,CDC_41401,CDC_496,CDC_5990,CDC_53081,Age(26-40),State_Count,Age(81-100),Female_Patient,CDC_4019,DGC_202,DGC_192,DGC_188,CPC_66,CPC_3995
count,500.0,500.0,500.0,500.0,500.000000,500.000000,500.000000,500.000000,500.000000,500.0,500.0,500.0,500.000000,500.000000
mean,0.0,0.0,0.0,0.0,1.358000,2.328000,15.298000,30.974000,3.384000,0.0,0.0,0.0,0.072000,0.066000
std,0.0,0.0,0.0,0.0,2.516048,2.472376,25.950956,50.249331,2.658322,0.0,0.0,0.0,0.258747,0.248531
min,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
25%,0.0,0.0,0.0,0.0,0.000000,1.000000,2.000000,4.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000
50%,0.0,0.0,0.0,0.0,0.000000,1.000000,6.000000,13.000000,3.000000,0.0,0.0,0.0,0.000000,0.000000
75%,0.0,0.0,0.0,0.0,2.000000,3.000000,17.000000,34.250000,5.000000,0.0,0.0,0.0,0.000000,0.000000
max,0.0,0.0,0.0,0.0,24.000000,19.000000,189.000000,344.000000,10.000000,0.0,0.0,0.0,1.000000,1.000000


In [194]:
NFraud_df[['CDC_41401', 'CDC_496', 'CDC_5990', 'CDC_53081', 'Age(26-40)',  'State_Count',  'Age(81-100)',  'Female_Patient', 'CDC_4019', 'DGC_202', 'DGC_192',
 'DGC_188', 'CPC_66', 'CPC_3995']].median()

CDC_41401          0.0
CDC_496            0.0
CDC_5990           0.0
CDC_53081          0.0
Age(26-40)         0.0
State_Count        1.0
Age(81-100)        6.0
Female_Patient    13.0
CDC_4019           3.0
DGC_202            0.0
DGC_192            0.0
DGC_188            0.0
CPC_66             0.0
CPC_3995           0.0
dtype: float64

In [59]:
Fraud_CM = Fraud_df.drop('Index', axis = 1).corr().unstack().drop_duplicates().\
to_frame().reset_index().rename(columns = {'level_0' : 'Feature1', 'level_1' : 'Feature2', 0 : 'Corr_Coef'})
Fraud_CM.loc[Fraud_CM.Corr_Coef > 0.6].sort_values(ascending = False, by = 'Corr_Coef').head(15)

,Feature1,Feature2,Corr_Coef
0,Male_Patient,Male_Patient,1.000000
914,Diabetes,IschemicHeart,0.999344
714,Heartfailure,IschemicHeart,0.998905
713,Heartfailure,Diabetes,0.998890
875,Depression,Diabetes,0.998363
670,Alzheimer,Diabetes,0.998023
876,Depression,IschemicHeart,0.998007
671,Alzheimer,IschemicHeart,0.997896
712,Heartfailure,Depression,0.997711
709,Heartfailure,KidneyDisease,0.997623


In [60]:
NFraud_CM = NFraud_df.drop('Index', axis = 1).corr().unstack().drop_duplicates().\
to_frame().reset_index().rename(columns = {'level_0' : 'Feature1', 'level_1' : 'Feature2', 0 : 'Corr_Coef'})
NFraud_CM.loc[NFraud_CM.Corr_Coef > 0.6].sort_values(ascending = False, by = 'Corr_Coef').head(15)

,Feature1,Feature2,Corr_Coef
0,Male_Patient,Male_Patient,1.000000
914,Diabetes,IschemicHeart,0.997403
714,Heartfailure,IschemicHeart,0.996281
713,Heartfailure,Diabetes,0.995883
61,Female_Patient,Age(61-80),0.993114
875,Depression,Diabetes,0.992857
63,Female_Patient,Race_White,0.992810
876,Depression,IschemicHeart,0.992658
4,Male_Patient,Age(61-80),0.992476
755,KidneyDisease,Diabetes,0.992390


# Market-basket Analysis (MBA)

In [36]:
pd.set_option('display.max_colwidth', 500)

In [34]:
df_N = NFraud_df[['Alzheimer',
       'Heartfailure', 'KidneyDisease', 'Cancer', 'ObstrPulmonary',
       'Depression', 'Diabetes', 'IschemicHeart', 'Osteoporasis',
       'rheumatoidarthritis', 'stroke', 'RenalDisease']].copy()
df_N = df_N.astype(bool)

df_F = Fraud_df[['Alzheimer',
       'Heartfailure', 'KidneyDisease', 'Cancer', 'ObstrPulmonary',
       'Depression', 'Diabetes', 'IschemicHeart', 'Osteoporasis',
       'rheumatoidarthritis', 'stroke', 'RenalDisease']].copy()

In [38]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(df_N, min_support=0.8, use_colnames=True)
MBA_chronic = association_rules(frequent_itemsets, metric="lift", min_threshold=0.7)

In [39]:
MBA_chronic

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Alzheimer),(Heartfailure),0.934543,0.968189,0.923328,0.987999,1.020461,0.018513,2.650704
1,(Heartfailure),(Alzheimer),0.968189,0.934543,0.923328,0.953665,1.020461,0.018513,1.412676
2,(KidneyDisease),(Alzheimer),0.936582,0.934543,0.902325,0.963423,1.030902,0.027048,1.789541
3,(Alzheimer),(KidneyDisease),0.934543,0.936582,0.902325,0.965525,1.030902,0.027048,1.839512
4,(Alzheimer),(Cancer),0.934543,0.822186,0.807912,0.864499,1.051464,0.039544,1.312273
...,...,...,...,...,...,...,...,...,...
33297,(Diabetes),"(KidneyDisease, Depression, rheumatoidarthritis, Alzheimer, Osteoporasis, Heartfailure, IschemicHeart, ObstrPulmonary)",0.978385,0.820147,0.819943,0.838058,1.021838,0.017524,1.110599
33298,(Osteoporasis),"(KidneyDisease, Depression, rheumatoidarthritis, Alzheimer, Diabetes, Heartfailure, IschemicHeart, ObstrPulmonary)",0.912316,0.837072,0.819943,0.898748,1.073681,0.056269,1.609141
33299,(Heartfailure),"(KidneyDisease, Depression, rheumatoidarthritis, Alzheimer, Diabetes, Osteoporasis, IschemicHeart, ObstrPulmonary)",0.968189,0.820147,0.819943,0.846883,1.032599,0.025886,1.174612
33300,(IschemicHeart),"(KidneyDisease, Depression, rheumatoidarthritis, Alzheimer, Diabetes, Osteoporasis, Heartfailure, ObstrPulmonary)",0.986338,0.820147,0.819943,0.831300,1.013599,0.011001,1.066115
